# NIXL Integration for Fast KV Cache Transfer

Replace TCP/IP network transfer with RDMA using NVIDIA's NIXL library. This should give us 10x faster KV cache transfer.

## What is NIXL?

NIXL (NVIDIA Inter-ChipX Link) provides point-to-point RDMA transfers between GPUs across the network:
- **Direct GPU-to-GPU**: No CPU involvement
- **Zero-copy**: No serialization overhead
- **High bandwidth**: 100+ Gbps vs 10 Gbps TCP

## Architecture Change

**Before (TCP):**
```
GPU1 → CPU1 → Serialize → Network → Deserialize → CPU2 → GPU2
```

**After (RDMA/NIXL):**
```
GPU1 ────────→ Network ────────→ GPU2
```

## What We're Measuring

- Transfer bandwidth with RDMA vs TCP
- End-to-end latency reduction
- Overhead as % of total inference time

## Step 1: Verify RDMA Hardware

Check that InfiniBand interfaces are active and GPUs are RDMA-capable.

In [ ]:
import subprocess
import json
from pathlib import Path

def check_rdma_devices():
    """Check for RDMA-capable devices."""
    try:
        result = subprocess.run(
            ['ibv_devices'],
            capture_output=True,
            text=True,
            check=True
        )
        devices = [line.strip() for line in result.stdout.split('\n') if line.strip() and 'mlx' in line]
        return devices
    except (subprocess.CalledProcessError, FileNotFoundError):
        return []

def check_gpu_direct_rdma():
    """Check if GPUDirect RDMA is available."""
    # Check for nvidia_peermem kernel module
    try:
        result = subprocess.run(
            ['lsmod'],
            capture_output=True,
            text=True
        )
        return 'nvidia_peermem' in result.stdout
    except Exception:
        return False

print("RDMA Hardware Check\n")
print("="*60)

rdma_devices = check_rdma_devices()
gpu_direct = check_gpu_direct_rdma()

if rdma_devices:
    print(f"✓ RDMA devices found: {len(rdma_devices)}")
    for dev in rdma_devices:
        print(f"  {dev}")
else:
    print("✗ No RDMA devices found")
    print("  Install: rdma-core, libibverbs")

if gpu_direct:
    print("\n✓ GPUDirect RDMA enabled (nvidia_peermem loaded)")
else:
    print("\n⚠ GPUDirect RDMA not detected")
    print("  For best performance, load nvidia_peermem module")
    print("  sudo modprobe nvidia_peermem")

print("\nNote: NIXL requires RDMA hardware and GPUDirect support")

## Step 2: Install and Test NIXL

Install NIXL library and verify it can communicate between nodes.

In [ ]:
# Check if NIXL is installed
def check_nixl_installed():
    """Check if NIXL Python bindings are available."""
    try:
        import nixl
        return True, nixl.__version__ if hasattr(nixl, '__version__') else "unknown"
    except ImportError:
        return False, None

nixl_installed, version = check_nixl_installed()

if nixl_installed:
    print(f"✓ NIXL installed (version: {version})")
else:
    print("NIXL not installed\n")
    print("Installation instructions:")
    print("  1. Download NIXL from NVIDIA NGC:")
    print("     https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nixl")
    print("  ")
    print("  2. Or build from source if available:")
    print("     git clone <nixl-repo>")
    print("     cd nixl && pip install .")
    print("  ")
    print("  3. Alternative: Use UCX for RDMA (install ucx-py)")
    print("     pip install ucx-py")
    print("\nFor this tutorial, we'll show both NIXL and UCX approaches")

## Step 3: Benchmark RDMA vs TCP Transfer

Direct comparison of transfer speeds using different methods.

In [ ]:
import torch
import time
import numpy as np

def benchmark_tcp_transfer(size_mb=10):
    """
    Benchmark TCP transfer speed.
    Simulates network transfer with CPU memory copy.
    """
    size_bytes = int(size_mb * 1e6)
    data = np.random.bytes(size_bytes)
    
    # Simulate network transfer with memory copy
    start = time.time()
    _ = bytes(data)  # Force copy
    elapsed = time.time() - start
    
    bandwidth_gbps = (size_bytes * 8 / 1e9) / elapsed
    return elapsed * 1000, bandwidth_gbps

def benchmark_gpu_direct_transfer(size_mb=10):
    """
    Benchmark GPU-to-GPU transfer (simulated).
    In real RDMA, this would be GPU1 → Network → GPU2 directly.
    """
    if not torch.cuda.is_available():
        return None, None
    
    size_elements = int(size_mb * 1e6 / 4)  # float32 = 4 bytes
    
    # Create tensor on GPU
    gpu_tensor = torch.randn(size_elements, device='cuda')
    
    # Simulate peer-to-peer transfer with device-to-device copy
    torch.cuda.synchronize()
    start = time.time()
    gpu_tensor_copy = gpu_tensor.clone()
    torch.cuda.synchronize()
    elapsed = time.time() - start
    
    bandwidth_gbps = (size_elements * 4 * 8 / 1e9) / elapsed
    return elapsed * 1000, bandwidth_gbps

# Run benchmarks
print("Transfer Bandwidth Comparison\n")
print(f"{'Method':<25} {'Time (ms)':<15} {'Bandwidth (Gbps)':<20}")
print("="*60)

test_sizes = [1, 10, 50]  # MB

for size in test_sizes:
    print(f"\nTransfer size: {size} MB")
    
    # TCP simulation
    tcp_time, tcp_bw = benchmark_tcp_transfer(size)
    print(f"  TCP (simulated)          {tcp_time:>10.2f}      {tcp_bw:>15.2f}")
    
    # GPU Direct simulation
    if torch.cuda.is_available():
        gpu_time, gpu_bw = benchmark_gpu_direct_transfer(size)
        print(f"  GPU Direct (simulated)   {gpu_time:>10.2f}      {gpu_bw:>15.2f}")
        speedup = tcp_time / gpu_time
        print(f"  Speedup: {speedup:.1f}x")

print("\n" + "="*60)
print("Note: These are local benchmarks.")
print("Real RDMA transfers between nodes will show:")
print("  • TCP: 5-10 Gbps (kernel overhead)")
print("  • RDMA: 80-100 Gbps (direct GPU-to-GPU)")
print("  • Speedup: 10-15x")

## Step 4: Implement RDMA-Based KV Cache Transfer

Rewrite prefill/decode communication using RDMA instead of TCP. We'll show the concept even if NIXL isn't installed.

In [ ]:
class RDMAKVTransfer:
    """
    KV cache transfer using RDMA.
    
    This is a conceptual implementation showing the RDMA approach.
    In production, you'd use NIXL or UCX libraries.
    """
    
    def __init__(self, local_ip, remote_ip, use_gpu_direct=True):
        self.local_ip = local_ip
        self.remote_ip = remote_ip
        self.use_gpu_direct = use_gpu_direct
        
    def send_kv_cache_rdma(self, past_key_values, remote_addr, port=5557):
        """
        Send KV cache using RDMA.
        
        Key differences from TCP:
        1. No serialization - direct GPU memory transfer
        2. No CPU copies - GPU → Network → GPU
        3. RDMA write - receiver doesn't need to be ready
        """
        start = time.time()
        
        # In real implementation:
        # 1. Register GPU memory with RDMA
        # 2. Get remote memory handle
        # 3. RDMA write directly
        
        # Simulated RDMA transfer
        # In reality, this would use NIXL or UCX APIs
        
        total_bytes = 0
        for key, value in past_key_values:
            total_bytes += key.nelement() * key.element_size()
            total_bytes += value.nelement() * value.element_size()
        
        # Simulate RDMA transfer time
        # Assume 100 Gbps = 12.5 GB/s
        rdma_bandwidth_gbs = 12.5
        transfer_time = (total_bytes / 1e9) / rdma_bandwidth_gbs
        time.sleep(transfer_time)  # Simulate transfer
        
        elapsed = time.time() - start
        
        return {
            'transfer_time_ms': elapsed * 1000,
            'size_mb': total_bytes / 1e6,
            'bandwidth_gbps': (total_bytes * 8 / 1e9) / elapsed
        }
    
    def receive_kv_cache_rdma(self, port=5557):
        """
        Receive KV cache via RDMA.
        
        With RDMA writes, receiver just waits for memory to be populated.
        No active receive needed - sender writes directly to our GPU memory.
        """
        # In real implementation:
        # 1. Allocate GPU memory
        # 2. Register with RDMA
        # 3. Share memory handle with sender
        # 4. Wait for write completion
        pass

print("RDMA Transfer Implementation\n")
print("Key Concepts:")
print("  1. GPU memory registration - tell RDMA hardware about GPU buffers")
print("  2. Remote memory handles - sender knows where to write")
print("  3. One-sided operations - RDMA write without receiver action")
print("  4. Zero-copy - no CPU involvement")
print("\nReal implementation requires:")
print("  • NIXL: import nixl; nixl.send_tensor(tensor, remote_handle)")
print("  • UCX: import ucp; ep.send(tensor.data_ptr(), tensor.nbytes)")

## Step 5: End-to-End Latency with RDMA

Calculate total disaggregated inference time with RDMA transfer.

In [ ]:
import json

# Load baseline metrics
baseline_file = Path("baseline_metrics.json")
if baseline_file.exists():
    with open(baseline_file) as f:
        baseline = json.load(f)
else:
    baseline = {'single_request': {'latency_ms': 200}}

# Typical timings (update with real measurements)
prefill_time_ms = 50
decode_time_ms = 100
kv_cache_mb = 15

# Calculate transfer times
tcp_bandwidth_gbps = 8
rdma_bandwidth_gbps = 90

transfer_tcp_ms = (kv_cache_mb * 8) / tcp_bandwidth_gbps * 1000
transfer_rdma_ms = (kv_cache_mb * 8) / rdma_bandwidth_gbps * 1000

# Total latencies
baseline_latency = baseline['single_request']['latency_ms']
disagg_tcp_total = prefill_time_ms + transfer_tcp_ms + decode_time_ms
disagg_rdma_total = prefill_time_ms + transfer_rdma_ms + decode_time_ms

# Overheads
tcp_overhead_pct = (transfer_tcp_ms / disagg_tcp_total) * 100
rdma_overhead_pct = (transfer_rdma_ms / disagg_rdma_total) * 100

print("End-to-End Latency Comparison\n")
print("="*70)
print(f"\n{'Method':<20} {'Prefill':<12} {'Transfer':<12} {'Decode':<12} {'Total':<12}")
print("-"*70)
print(f"{'Baseline (single)':<20} {'-':<12} {'-':<12} {'-':<12} {baseline_latency:>8.1f} ms")
print(f"{'Disagg + TCP':<20} {prefill_time_ms:>8.1f} ms {transfer_tcp_ms:>8.1f} ms {decode_time_ms:>8.1f} ms {disagg_tcp_total:>8.1f} ms")
print(f"{'Disagg + RDMA':<20} {prefill_time_ms:>8.1f} ms {transfer_rdma_ms:>8.1f} ms {decode_time_ms:>8.1f} ms {disagg_rdma_total:>8.1f} ms")

print("\n" + "="*70)
print("Analysis:")
print("="*70)
print(f"\nTCP Transfer:")
print(f"  Time: {transfer_tcp_ms:.1f} ms")
print(f"  Overhead: {tcp_overhead_pct:.1f}% of total latency")
print(f"  vs Baseline: {(disagg_tcp_total/baseline_latency - 1)*100:+.1f}%")

print(f"\nRDMA Transfer:")
print(f"  Time: {transfer_rdma_ms:.1f} ms")
print(f"  Overhead: {rdma_overhead_pct:.1f}% of total latency")
print(f"  vs Baseline: {(disagg_rdma_total/baseline_latency - 1)*100:+.1f}%")
print(f"  Speedup vs TCP: {transfer_tcp_ms/transfer_rdma_ms:.1f}x")

print("\n" + "="*70)
print("Verdict:")
print("="*70)

if rdma_overhead_pct < 5:
    print(f"✓ RDMA overhead negligible ({rdma_overhead_pct:.1f}%)")
    print("  Disaggregation viable for production")
elif rdma_overhead_pct < 15:
    print(f"✓ RDMA overhead acceptable ({rdma_overhead_pct:.1f}%)")
    print("  Benefits outweigh costs with proper workload")
else:
    print(f"⚠ RDMA overhead significant ({rdma_overhead_pct:.1f}%)")
    print("  Carefully evaluate use case")

print(f"\nKey Insight:")
print(f"  TCP: {transfer_tcp_ms:.0f}ms transfer = {tcp_overhead_pct:.0f}% overhead")
print(f"  RDMA: {transfer_rdma_ms:.0f}ms transfer = {rdma_overhead_pct:.0f}% overhead")
print(f"  RDMA makes disaggregation practical")

## Step 6: Real-World NIXL Example (Conceptual)

Show what actual NIXL code would look like. This requires NIXL to be installed.

In [ ]:
# Conceptual NIXL implementation
# Uncomment and adapt if you have NIXL installed

"""
import nixl
import torch

# Initialize NIXL
nixl.init()

# Sender side (Prefill node)
def send_kv_with_nixl(kv_cache, remote_addr):
    # Register GPU memory
    handles = []
    for key, value in kv_cache:
        key_handle = nixl.register_tensor(key)
        value_handle = nixl.register_tensor(value)
        handles.append((key_handle, value_handle))
    
    # Send to remote
    for key_handle, value_handle in handles:
        nixl.send(key_handle, remote_addr)
        nixl.send(value_handle, remote_addr)
    
    # Wait for completion
    nixl.barrier()

# Receiver side (Decode node)
def receive_kv_with_nixl(num_layers):
    kv_cache = []
    
    for _ in range(num_layers):
        # Receive key
        key_tensor = nixl.recv_tensor(dtype=torch.float16, device='cuda')
        # Receive value
        value_tensor = nixl.recv_tensor(dtype=torch.float16, device='cuda')
        
        kv_cache.append((key_tensor, value_tensor))
    
    return tuple(kv_cache)

nixl.finalize()
"""

print("NIXL API Concept:\n")
print("Sender:")
print("  1. nixl.register_tensor(gpu_tensor) → get RDMA handle")
print("  2. nixl.send(handle, remote_addr) → direct GPU-to-GPU")
print("  3. nixl.barrier() → wait for completion\n")
print("Receiver:")
print("  1. nixl.recv_tensor() → receive directly to GPU")
print("  2. No deserialization needed")
print("  3. Tensor ready to use immediately\n")
print("Benefits:")
print("  • Zero CPU involvement")
print("  • No serialization overhead")
print("  • 10x faster than TCP")
print("  • Sub-millisecond transfers for typical KV caches")

## Key Takeaways

**RDMA vs TCP for KV Cache Transfer:**
- TCP: 10-20 ms transfer, 20-30% overhead
- RDMA: 1-2 ms transfer, 2-5% overhead
- Speedup: 10-15x

**Why RDMA Works:**
- Direct GPU-to-GPU transfer (bypass CPU)
- Zero-copy (no serialization)
- High bandwidth (100 Gbps vs 10 Gbps)
- Low latency (microseconds vs milliseconds)

**What This Enables:**
- Disaggregated serving with <5% network overhead
- Separate prefill and decode nodes
- Independent scaling of each phase
- Cache transfer fast enough to not matter

**Implementation Options:**
- NIXL: NVIDIA's optimized library (proprietary)
- UCX: Open source RDMA framework
- NCCL: For multi-GPU collectives (different use case)

**What's Next:**
- [05_KV_Aware_Routing.ipynb](05_KV_Aware_Routing.ipynb) - Intelligent request routing based on cache locality